In [1]:
import sys,subprocess
pkgs=["autogluon.tabular==1.4.0","catboost==1.2.8","tqdm"]
subprocess.check_call([sys.executable,"-m","pip","install","-q"]+pkgs)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.3/487.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.0/71.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.0/278.0 kB 21.0 MB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
category-encoders 2.7.0 requires scikit-learn<1.6.0,>=1.0.0, but you have scikit-learn 1.7.2 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
sklearn-compat 0.1.3 requires scikit-learn<1.7,>=1.2, but you have scikit-learn 1.7.2 which is incompatible.


0

In [ ]:
#!pip install "protobuf<5" --force-reinstall -q


In [3]:
# Глобальный конфиг для всей тетрадки
# Здесь можно быстро менять датасет, тип задачи, пути к файлам и настройки картинок.

CONFIG = {
    # === основные настройки задачи ===
    "task_type": "regression",          # "regression" | "binary" | "multiclass"
    "metric": "rmse",                   # метрика для AutoGluon / CatBoost / пр.

    # папка с табличными данными и имена файлов
    "data_dir": ".",                    # где лежат train / test
    "train_csv": "/kaggle/input/petfinder-pawpularity-score/train.csv",
    "test_csv": "/kaggle/input/petfinder-pawpularity-score/test.csv",
    "sep": ",",                         # разделитель в csv/tsv
    "target_column": "Pawpularity",          # имя таргета
    "id_column": "Id",                  # id-колонка в test (можно None)

    # === настройки AutoML / времени ===
    "time_limit_sec": 1800,             # время на medium-модель (сек)
    "cat_time_limit_sec": 1800,         # время на CatBoost-only
    "heavy_time_limit_sec": 7200,       # время на heavy-режим
    "autogluon_presets": "medium_quality_faster_train",

    # ---------- images (CLIP) ---------
    # папка с train-картинками (если используем image-фичи)
    "train_images_dir": "/kaggle/input/petfinder-pawpularity-score/train",  # например "/kaggle/input/petfinder-pawpularity-score/train"

    # папка с test-картинками (если используем image-фичи)
    "test_images_dir": "/kaggle/input/petfinder-pawpularity-score/test",   # например "/kaggle/input/petfinder-pawpularity-score/test"

    # Если в таблице image_id уже содержит ".jpg", поставь image_ext = ""
    # расширение файлов картинок (если в таблице только id без расширения)
    "image_ext": ".jpg",

    # колонка в таблице с именем файла картинки
    # Обычно совпадает с id, к которому дописывается image_ext.
    "file_names_column": "Id",

    # размер батча при обработке картинок
    # Чем больше батч, тем быстрее, но тем больше память видеокарты/CPU.
    "batch_size": 32,

    # имя CLIP-модели в HuggingFace, которая используется для извлечения image-эмбеддингов.
    "clip_model_name": "openai/clip-vit-base-patch32",
}


In [4]:
import pandas as pd
import numpy as np
from pathlib import Path
from autogluon.features.generators import AutoMLPipelineFeatureGenerator
from catboost import CatBoostRegressor,Pool
from catboost.utils import get_gpu_device_count
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from tqdm.auto import tqdm

from pathlib import Path
import pandas as pd

# базовая папка с данными из CONFIG
path = Path(CONFIG.get("data_dir", "."))


In [5]:
# читаем train / test согласно CONFIG
train_path = path / CONFIG.get("train_csv", "train.csv")
test_path  = path / CONFIG.get("test_csv", "test.csv")

train = pd.read_csv(train_path, sep=CONFIG.get("sep", ","))
test = pd.read_csv(test_path,  sep=CONFIG.get("sep", ","))


In [6]:
train.columns

Index(['Id', 'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
       'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur',
       'Pawpularity'],
      dtype='object')

In [7]:
test

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur
0,4128bae22183829d2b5fea10effdb0c3,1,0,1,0,0,1,1,0,0,1,0,1
1,43a2262d7738e3d420d453815151079e,0,1,0,0,0,0,1,1,0,0,0,0
2,4e429cead1848a298432a0acad014c9d,0,0,0,1,0,1,1,1,0,1,1,1
3,80bc3ccafcc51b66303c2c263aa38486,1,0,1,0,0,0,0,0,0,0,1,0
4,8f49844c382931444e68dffbe20228f4,1,1,1,0,1,1,0,1,0,1,1,0
5,b03f7041962238a7c9d6537e22f9b017,0,0,1,1,1,1,1,1,1,0,1,0
6,c978013571258ed6d4637f6e8cc9d6a3,1,0,0,0,1,1,0,1,0,1,1,1
7,e0de453c1bffc20c22b072b34b54e50f,1,0,1,0,0,0,0,0,1,0,0,1


In [8]:
# выбираем таргет и базовые фичи из CONFIG
target_col = CONFIG.get("target_column", "target")  # или как называется таргет в новом датасете

feature_cols = [c for c in train.columns if c != target_col and c in test.columns]

X = train[feature_cols].copy()
y = train[target_col].values
X_test = test[feature_cols].copy()


In [9]:
# ==== РУЧНЫЕ (КАСТОМНЫЕ) ФИЧИ ====
# В этой ячейке можно руками дописывать любые фичи на табличных данных.
# Важно: делайте одинаковые преобразования для train (X) и test (X_test).

def add_manual_features(X: pd.DataFrame, X_test: pd.DataFrame):
    """
    Здесь можно руками добавить любые фичи на основе табличных данных.
    Пример:
        if "col1" in X.columns and "col2" in X.columns:
            X["col1_div_col2"] = X["col1"] / (X["col2"] + 1)
            X_test["col1_div_col2"] = X_test["col1"] / (X_test["col2"] + 1)

    Не забывайте синхронизировать преобразования для train и test.
    """
    # --- пример (можно удалить/изменить) ---
    # if "col1" in X.columns and "col2" in X.columns:
    #     X["col1_div_col2"] = X["col1"] / (X["col2"] + 1)
    #     X_test["col1_div_col2"] = X_test["col1"] / (X_test["col2"] + 1)
    # ---------------------------------------
    return X, X_test


X, X_test = add_manual_features(X, X_test)
print("После ручных фичей:", X.shape, X_test.shape)


После ручных фичей: (9912, 13) (8, 13)


In [10]:
# ==== ИМЕНОВЫЕ ФИЧИ ИЗ КАРТИНОК (CLIP) ====
# Если в таблице есть колонка с именем файла картинки, можно добавить
# к табличным фичам эмбеддинги, посчитанные CLIP-моделью.

import os

import torch
from PIL import Image
from transformers import CLIPModel, CLIPProcessor
from tqdm.auto import tqdm

def _normalize_image_name(name: str, default_ext: str | None = None) -> str:
    """
    Приводим имя файла к нормальному виду:
    - убираем пробелы/переводы строк
    - берём только basename пути
    - если нет расширения, добавляем default_ext или CONFIG["image_ext"]
    """
    name = str(name).strip()
    base = os.path.basename(name)
    root, ext = os.path.splitext(base)
    if ext == "":
        if default_ext is None:
            default_ext = CONFIG.get("image_ext", "")
        return root + (default_ext or "")
    return root + ext


def process_images(file_paths, model, processor, device, name):
    """
    Считает эмбеддинги для списка путей к картинкам одной CLIP-моделью.
    Использует CLIPProcessor + CLIPModel.get_image_features.
    """
    batch_size = CONFIG.get("batch_size", 32)

    collect_embs = []
    if not file_paths:
        return pd.DataFrame()

    for start in tqdm(
        range(0, len(file_paths), batch_size),
        total=(len(file_paths) + batch_size - 1) // batch_size,
        desc=f"Images {name} process..."
    ):
        batch_paths = file_paths[start:start + batch_size]
        images = []
        for path in batch_paths:
            try:
                img = Image.open(path).convert("RGB")
            except Exception:
                # если не удалось открыть картинку — используем заглушку
                img = Image.new("RGB", (224, 224), color=0)
            images.append(img)

        with torch.no_grad():
            inputs = processor(images=images, return_tensors="pt", padding=True).to(device)
            embs = model.get_image_features(**inputs)

            # L2-нормализация по строкам (num_samples x dim)
            embs = embs / (embs.norm(dim=-1, keepdim=True) + 1e-12)

        collect_embs.extend(embs.cpu().numpy().tolist())

    if not collect_embs:
        # На всякий случай, чтобы не упасть на пустом списке
        return pd.DataFrame()

    n_feat = len(collect_embs[0])
    return pd.DataFrame(collect_embs, columns=[f"emb_{i}" for i in range(n_feat)])


def images_features(X: pd.DataFrame, test_df: pd.DataFrame | None):
    """
    Добавляет к табличным фичам эмбеддинги картинок, посчитанные CLIP-моделью от OpenAI
    (через HuggingFace Transformers, только image-encoder).

    Ожидается, что в CONFIG заданы:
      - 'file_names_column'   — колонка с именами файлов
      - 'train_images_dir'    — папка с train-картинками
      - 'test_images_dir'     — папка с test-картинками
      - 'batch_size'          — размер батча для инференса
      - 'clip_model_name'     — имя CLIP-модели в HuggingFace (по умолчанию ViT-L/14)
    """
    file_name_column = CONFIG["file_names_column"]

    if file_name_column not in X.columns:
        raise KeyError(f"CONFIG['file_names_column']={file_name_column!r} нет в train (X)")

    if test_df is not None and file_name_column not in test_df.columns:
        raise KeyError(f"CONFIG['file_names_column']={file_name_column!r} нет в test (X_test)")

    # ----------------- CLIP BACKBONE (image encoder only) -----------------
    if hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
        device = "mps"
    else:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    clip_model_name = CONFIG.get("clip_model_name", "openai/clip-vit-large-patch14")
    # Загружаем модель и процессор CLIP
    model = CLIPModel.from_pretrained(clip_model_name)
    processor = CLIPProcessor.from_pretrained(clip_model_name)

    model = model.to(device)
    model.eval()

    # ----------------- TRAIN IMAGES -----------------
    train_file_paths = [
        os.path.join(
            CONFIG["train_images_dir"],
            _normalize_image_name(file_name, CONFIG.get("image_ext", ""))
        )
        for file_name in X[file_name_column]
    ]
    train_images_part = process_images(train_file_paths, model, processor, device, name="train_clip")
    train_images_part.index = X.index
    X = pd.concat([X, train_images_part], axis=1)

    # ----------------- TEST IMAGES ------------------
    if test_df is not None:
        test_file_paths = [
            os.path.join(
                CONFIG["test_images_dir"],
                _normalize_image_name(file_name, CONFIG.get("image_ext", ""))
            )
            for file_name in test_df[file_name_column]
        ]
        test_images_part = process_images(test_file_paths, model, processor, device, name="test_clip")
        test_images_part.index = test_df.index
        test_df = pd.concat([test_df, test_images_part], axis=1)

    # (опционально) выбрасываем колонку с именем файла, чтобы она не ушла в бустинги
    for df in (X, test_df):
        if df is not None and file_name_column in df.columns:
            df.drop(columns=[file_name_column], inplace=True)

    return X, test_df


# Если папка с картинками задана в CONFIG — считаем эмбеддинги и добавляем их к X/X_test
if CONFIG.get("train_images_dir"):
    print("Images process starts...")
    X, X_test = images_features(X, X_test)
    print("После image-фичей:", X.shape, X_test.shape)
else:
    print("CONFIG['train_images_dir'] пуст — image-фичи не считаем.")


2025-11-18 22:27:25.182701: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763504845.370702      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763504845.422571      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

Images train_clip process...:   0%|          | 0/310 [00:00<?, ?it/s]

Images test_clip process...:   0%|          | 0/1 [00:00<?, ?it/s]

После image-фичей: (9912, 524) (8, 524)


In [12]:
print("До автофичей:", X.shape)          # (41105, N_исходных_фич)
print("Число исходных фич:", X.shape[1])


До автофичей: (9912, 524)
Число исходных фич: 524


In [13]:
from autogluon.features.generators import AutoMLPipelineFeatureGenerator

fg = AutoMLPipelineFeatureGenerator(
    enable_numeric_features=True,
    enable_categorical_features=True,
    enable_datetime_features=True,
    enable_text_special_features=True,
    enable_text_ngram_features=True,
    enable_raw_text_features=False,
)

X_tr_feat = fg.fit_transform(X=X, y=y)
X_te_feat = fg.transform(X_test)

print(X_tr_feat.shape, X_te_feat.shape)
print(X_tr_feat.columns[:30])



(9912, 524) (8, 524)
Index(['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group',
       'Collage', 'Human', 'Occlusion', 'Info', 'Blur', 'emb_0', 'emb_1',
       'emb_2', 'emb_3', 'emb_4', 'emb_5', 'emb_6', 'emb_7', 'emb_8', 'emb_9',
       'emb_10', 'emb_11', 'emb_12', 'emb_13', 'emb_14', 'emb_15', 'emb_16',
       'emb_17'],
      dtype='object')


In [14]:
print("После автофичей:", X_tr_feat.shape)   # (41105, 916)
print("Число фич после генерации:", X_tr_feat.shape[1])


После автофичей: (9912, 524)
Число фич после генерации: 524


In [15]:
from catboost.utils import get_gpu_device_count
import catboost, sys
print("python:", sys.executable)
print("catboost version:", catboost.__version__)
print("GPU devices seen by catboost:", get_gpu_device_count())


python: /usr/bin/python3
catboost version: 1.2.8
GPU devices seen by catboost: 2


In [16]:
import numpy as np

cat_cols = [
    c for c in X_tr_feat.columns
    if str(X_tr_feat[c].dtype) in ("category", "object")
]

for c in cat_cols:
    X_tr_feat[c] = X_tr_feat[c].astype(str).replace({"nan": "missing"}).fillna("missing")
    if c in X_te_feat.columns:
        X_te_feat[c] = X_te_feat[c].astype(str).replace({"nan": "missing"}).fillna("missing")

print("Категориальных колонок:", len(cat_cols))


Категориальных колонок: 0


In [17]:
cat_features = [
    c for c in X_tr_feat.columns
    if str(X_tr_feat[c].dtype) in ("object",)  # теперь всё строковое
]
print(len(cat_features), "cat_features")


0 cat_features


In [18]:
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_squared_error
import numpy as np

idx = np.arange(len(X_tr_feat))
np.random.seed(42)
np.random.shuffle(idx)
small_idx = idx[:1000]

X_small = X_tr_feat.iloc[small_idx]
y_small = y[small_idx]

train_pool = Pool(X_small, y_small, cat_features=cat_features)

model_gpu_test = CatBoostRegressor(
    loss_function="RMSE",
    eval_metric="RMSE",
    iterations=300,
    learning_rate=0.05,
    depth=6,
    task_type="GPU",
    devices="0",
    verbose=50,
    random_seed=42,
)

print("=== GPU sanity fit (300 iters on 1000 rows) ===")
model_gpu_test.fit(train_pool)


=== GPU sanity fit (300 iters on 1000 rows) ===
0:	learn: 20.5999128	total: 147ms	remaining: 43.9s
50:	learn: 15.9621574	total: 615ms	remaining: 3s
100:	learn: 13.6811772	total: 995ms	remaining: 1.96s
150:	learn: 11.5693277	total: 1.3s	remaining: 1.28s
200:	learn: 9.8029428	total: 1.6s	remaining: 789ms
250:	learn: 8.2915987	total: 1.9s	remaining: 372ms
299:	learn: 7.0792625	total: 2.21s	remaining: 0us


In [19]:
import pandas as pd
import numpy as np

for c in X_tr_feat.columns:
    if X_tr_feat[c].dtype == "object":
        X_tr_feat[c] = X_tr_feat[c].fillna("missing").astype(str)
        if c in X_te_feat.columns:
            X_te_feat[c] = X_te_feat[c].fillna("missing").astype(str)
    else:
        X_tr_feat[c] = pd.to_numeric(X_tr_feat[c], errors="coerce")
        if c in X_te_feat.columns:
            X_te_feat[c] = pd.to_numeric(X_te_feat[c], errors="coerce")

cat_features = [
    i for i, c in enumerate(X_tr_feat.columns)
    if X_tr_feat[c].dtype == "object"
]

print("cat_features count:", len(cat_features))

cat_features count: 0


In [20]:
'''надо поменять вообще было функцию на MAE для этой соревы но я проебался малек'''

'надо поменять вообще было функцию на MAE для этой соревы но я проебался малек'

In [21]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor, Pool
from tqdm.auto import tqdm
import numpy as np

def rmse_func(y_true, y_pred):
    return mean_squared_error(y_true, y_pred) ** 0.5

N_SPLITS = 5
N_ITER = 3000
DEPTH = 8
OD_WAIT = 300
TASK_TYPE = "GPU"

X = X_tr_feat
y_arr = y

kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

oof = np.zeros(len(X))
test_pred = np.zeros(len(X_te_feat))
scores = []

for fold, (tr_idx, val_idx) in enumerate(tqdm(kf.split(X), total=N_SPLITS, desc="Folds"), 1):
    X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
    y_tr, y_val = y_arr[tr_idx], y_arr[val_idx]

    train_pool = Pool(X_tr, y_tr, cat_features=cat_features)
    val_pool = Pool(X_val, y_val, cat_features=cat_features)

    params = dict(
        loss_function="RMSE",
        eval_metric="RMSE",
        iterations=N_ITER,
        learning_rate=0.01,
        depth=DEPTH,
        l2_leaf_reg=3.0,
        random_seed=42 + fold,
        verbose=200,
        od_type="Iter",
        od_wait=OD_WAIT,
        task_type=TASK_TYPE,
    )

    if TASK_TYPE == "GPU":
        params["devices"] = "0"

    print(f"\n=== Fold {fold}/{N_SPLITS} ===")
    model = CatBoostRegressor(**params)
    model.fit(train_pool, eval_set=val_pool, use_best_model=True)

    val_pred = model.predict(X_val)
    oof[val_idx] = val_pred
    rmse = rmse_func(y_val, val_pred)
    scores.append(rmse)
    print(f"Fold {fold} RMSE: {rmse:.5f}")

    bs = 1024
    fold_test = np.zeros(len(X_te_feat))
    for s in tqdm(range(0, len(X_te_feat), bs), leave=False, desc=f"Predict fold {fold}"):
        e = s + bs
        fold_test[s:e] = model.predict(X_te_feat.iloc[s:e])
    test_pred += fold_test / N_SPLITS

oof_rmse = rmse_func(y_arr, oof)
print("\nfold_rmse:", scores)
print("mean_rmse:", np.mean(scores))
print("oof_rmse:", oof_rmse)

Folds:   0%|          | 0/5 [00:00<?, ?it/s]


=== Fold 1/5 ===
0:	learn: 20.4522336	test: 21.0004615	best: 21.0004615 (0)	total: 213ms	remaining: 10m 38s
200:	learn: 17.9462165	test: 19.2109429	best: 19.2109429 (200)	total: 20.7s	remaining: 4m 47s
400:	learn: 16.9086892	test: 18.7397518	best: 18.7397518 (400)	total: 40.2s	remaining: 4m 20s
600:	learn: 16.2093929	test: 18.5461953	best: 18.5461953 (600)	total: 59.4s	remaining: 3m 57s
800:	learn: 15.6517869	test: 18.4435377	best: 18.4435377 (800)	total: 1m 18s	remaining: 3m 35s
1000:	learn: 15.1427062	test: 18.3765396	best: 18.3765396 (1000)	total: 1m 37s	remaining: 3m 15s
1200:	learn: 14.6678651	test: 18.3274354	best: 18.3272006 (1199)	total: 1m 57s	remaining: 2m 55s
1400:	learn: 14.2334530	test: 18.2987755	best: 18.2987755 (1400)	total: 2m 16s	remaining: 2m 36s
1600:	learn: 13.8428466	test: 18.2687638	best: 18.2687638 (1600)	total: 2m 36s	remaining: 2m 16s
1800:	learn: 13.4938830	test: 18.2518815	best: 18.2518815 (1800)	total: 2m 55s	remaining: 1m 56s
2000:	learn: 13.1561247	test:

KeyboardInterrupt: 

In [33]:
import numpy as np
import pandas as pd

# предполагаем, что есть:
# - test (оригинальный тестовый df)
# - test_pred (предсказания из k-fold)
# - target_col (имя таргета, как в train)

id_col = "id"  # или "ID", или другое
for c in ["id", "ID", "Id"]:
    if c in test.columns:
        id_col = c
        break

if id_col is not None:
    sub = pd.DataFrame({
        id_col: test[id_col].values,
        target_col: test_pred
    })
else:
    sub = pd.DataFrame({
        "id": np.arange(len(test_pred)),
        target_col: test_pred
    })

sub.to_csv("submission_catboost_kfold_autofeat_gpu.csv", index=False)
sub.head()

,id,target
0,21472,3.420300
1,9837,3.249490
2,41791,3.861935
3,18441,3.117375
4,49348,2.765881


In [22]:
train_ag = X_tr_feat.copy()
train_ag[target_col] = y

test_ag = X_te_feat.copy()

print(train_ag.shape, test_ag.shape)
train_ag.head()

(9912, 525) (8, 524)


,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,...,emb_503,emb_504,emb_505,emb_506,emb_507,emb_508,emb_509,emb_510,emb_511,Pawpularity
0,0,1,1,1,0,0,1,0,0,0,...,-0.024226,-0.033439,0.001547,-0.030126,0.010424,0.003675,0.050864,0.018073,0.016620,63
1,0,1,1,0,0,0,0,0,0,0,...,-0.001851,0.015791,-0.003030,-0.043720,-0.037799,-0.004829,0.033755,-0.034391,-0.000073,42
2,0,1,1,1,0,0,0,0,1,1,...,-0.024429,-0.050044,-0.004027,-0.035109,0.028670,0.016728,-0.030991,0.018655,0.028889,28
3,0,1,1,1,0,0,0,0,0,0,...,-0.024385,-0.022259,-0.007464,-0.027787,0.006609,-0.015715,0.050974,-0.004093,-0.000825,15
4,0,0,0,1,0,0,1,0,0,0,...,-0.007597,-0.032129,-0.024223,-0.025436,0.030226,0.000809,-0.030022,-0.015488,-0.017500,72


In [24]:
from autogluon.tabular import TabularPredictor


In [ ]:
predictor = TabularPredictor(
    label=target_col,
    problem_type=CONFIG.get("task_type", "regression"),
    eval_metric=CONFIG.get("metric", "rmse")
).fit(
    train_data=train_ag,
    time_limit=CONFIG.get("time_limit_sec", 1800),  # можно 600/900/3600 по ситуации
    presets="medium_quality_faster_train",
    ag_args_fit={"num_gpus": 1},
)


No path specified. Models will be saved in: "AutogluonModels/ag-20251118_224058"
Preset alias specified: 'medium_quality_faster_train' maps to 'medium_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.11.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Nov 10 10:07:59 UTC 2024
CPU Count:          4
Memory Avail:       28.61 GB / 31.35 GB (91.3%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Presets specified: ['medium_quality_faster_train']
Using hyperparameters preset: hyperparameters='default'
Beginning AutoGluon training ... Time limit = 1800s
AutoGluon will save models to "/kaggle/working/AutogluonModels/ag-20251118_224058"
Train Data Rows:    9912
Train Data Columns: 524
Label Column:       Pawpularity
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeature

In [38]:
print('working')

working


In [ ]:
''' если нужна модель тяжелее хотим presets="best_quality" '''

'''лидерборд'''
'''
leaderboard = predictor.leaderboard(train_ag, silent=True)
leaderboard
'''

'''предсказания и сабмит'''

'''
id_col = "id"  # или "ID", или другое имя, если нужно

ag_pred = predictor.predict(test_ag)

sub_ag = test[[id_col]].copy()
sub_ag[target_col] = ag_pred

sub_ag.to_csv("submission_autogluon_autofeat.csv", index=False)
sub_ag.head()
'''



In [37]:
leaderboard_cat = predictor_cat.leaderboard(train_ag, silent=True)
leaderboard_cat

ag_train_pred = predictor_cat.predict(train_ag)
ag_test_pred = predictor_cat.predict(test_ag)


NameError: name 'predictor_cat' is not defined

In [ ]:
"глюон тока с катбустом"

predictor_cat = TabularPredictor(
    label=target_col,
    problem_type=CONFIG.get("task_type", "regression"),
    eval_metric=CONFIG.get("metric", "rmse"),
).fit(
    train_data=train_ag,
    time_limit=CONFIG.get("cat_time_limit_sec", 1800),  # можно уменьшить/увеличить
    hyperparameters={
        'CAT': {}   # включаем только CatBoost-модели
    },
    ag_args_fit={"num_gpus": 1},
)


In [36]:
id_col = "id"  # или "ID", или другое имя, если нужно

ag_pred = predictor.predict(test_ag)

sub_ag = test[[id_col]].copy()
sub_ag[target_col] = ag_pred

sub_ag.to_csv("submission_autogluon_autofeat_id_target.csv", index=False)
sub_ag.head()


,id,target
0,21472,3.371200
1,9837,3.245652
2,41791,4.003885
3,18441,3.178790
4,49348,2.697172


In [ ]:
ag_pred = predictor.predict(test_ag)

sub_ag = pd.DataFrame({target_col: ag_pred})
sub_ag.to_csv("submission_autogluon_autofeat_only_id.csv", index=False)
sub_ag.head()

In [39]:
from autogluon.tabular import TabularPredictor

train_ag = X_tr_feat.copy()
train_ag[target_col] = y
test_ag = X_te_feat.copy()

train_ag.shape, test_ag.shape

((41105, 917), (9276, 916))

In [40]:
predictor_cat = TabularPredictor(
    label=target_col,
    problem_type=CONFIG.get("task_type", "regression"),
    eval_metric=CONFIG.get("metric", "rmse"),
).fit(
    train_data=train_ag,
    time_limit=CONFIG.get("cat_time_limit_sec", 1800),  # можно уменьшить/увеличить
    hyperparameters={
        'CAT': {}   # включаем только CatBoost-модели
    },
    ag_args_fit={"num_gpus": 1},
)


No path specified. Models will be saved in: "AutogluonModels/ag-20251116_190043"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #52~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Mon Dec  9 15:00:52 UTC 2
CPU Count:          62
Memory Avail:       427.80 GB / 503.46 GB (85.0%)
Disk Space Avail:   127.20 GB / 130.00 GB (97.8%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='extreme' : New in v1.4: Massively better than 'best' on datasets <30000 samples by using new models meta-learned on https://tabarena.ai: TabPFNv2, TabICL, Mitra, and TabM. Absolute best accuracy. Requires a GPU. Recommended 64 GB CPU memory 

In [41]:
leaderboard_cat = predictor_cat.leaderboard(train_ag, silent=True)
leaderboard_cat


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost,-1.003955,-0.981936,root_mean_squared_error,0.091085,0.044779,10.768425,0.091085,0.044779,10.768425,1,True,1
1,WeightedEnsemble_L2,-1.003955,-0.981936,root_mean_squared_error,0.093464,0.045044,10.770169,0.002378,0.000265,0.001745,2,True,2


In [42]:
ag_train_pred = predictor_cat.predict(train_ag)
ag_test_pred = predictor_cat.predict(test_ag)


In [ ]:
'''blend'''

In [46]:
from catboost import CatBoostRegressor
import numpy as np

Z_train = np.vstack([
    oof,                    # OOF из твоего KFold CatBoost
    ag_train_pred.values    # предсказания AutoGluon на train
]).T

Z_test = np.vstack([
    test_pred,              # предсказания твоего KFold CatBoost на test
    ag_test_pred.values     # предсказания AutoGluon на test
]).T

meta_model = CatBoostRegressor(
    loss_function="RMSE",
    eval_metric="RMSE",
    iterations=2000,
    learning_rate=0.03,
    depth=6,
    l2_leaf_reg=3.0,
    random_seed=777,
    verbose=100,
    task_type="CPU",   # можно сменить на "GPU", devices="0"
)

meta_model.fit(Z_train, y)

blend_oof = meta_model.predict(Z_train)
blend_rmse = rmse_func(y, blend_oof)
print("blend RMSE:", blend_rmse)

blend_test = meta_model.predict(Z_test)


0:	learn: 1.1803559	total: 16.7ms	remaining: 33.3s
100:	learn: 0.9898654	total: 784ms	remaining: 14.7s
200:	learn: 0.9848862	total: 1.65s	remaining: 14.8s
300:	learn: 0.9827911	total: 2.56s	remaining: 14.4s
400:	learn: 0.9804082	total: 3.36s	remaining: 13.4s
500:	learn: 0.9781054	total: 4.17s	remaining: 12.5s
600:	learn: 0.9761623	total: 4.79s	remaining: 11.2s
700:	learn: 0.9744053	total: 5.1s	remaining: 9.45s
800:	learn: 0.9728192	total: 5.49s	remaining: 8.22s
900:	learn: 0.9713416	total: 5.91s	remaining: 7.21s
1000:	learn: 0.9699321	total: 6.33s	remaining: 6.32s
1100:	learn: 0.9686531	total: 6.74s	remaining: 5.5s
1200:	learn: 0.9674257	total: 7.16s	remaining: 4.76s
1300:	learn: 0.9662867	total: 7.59s	remaining: 4.08s
1400:	learn: 0.9651149	total: 8.02s	remaining: 3.43s
1500:	learn: 0.9640560	total: 8.43s	remaining: 2.8s
1600:	learn: 0.9631643	total: 8.83s	remaining: 2.2s
1700:	learn: 0.9622188	total: 9.25s	remaining: 1.63s
1800:	learn: 0.9613573	total: 9.67s	remaining: 1.07s
1900:	le

In [44]:
# если в test есть id-колонка
id_col = "id"  # или своё имя

sub_blend = test[[id_col]].copy()
sub_blend[target_col] = blend_test
sub_blend.to_csv("submission_blend_catboost_meta.csv", index=False)
sub_blend.head()



In [ ]:
'''далее улучшение всего вышеперечисленного (потяжелее модели) + маленько иного'''
'''почти с нуля все'''

In [48]:
DATA_DIR = Path(CONFIG.get("data_dir", "."))

TRAIN_PATH = DATA_DIR / "train.tsv"   # или train.csv
TEST_PATH  = DATA_DIR / "test.tsv"    # или test.csv
SEP = CONFIG.get("sep", ",")            # "\t" для tsv, "," для csv

TARGET_COL = CONFIG.get("target_column", "target")            # имя таргета
ID_COL = CONFIG.get("id_column", "id")                         # имя id-колонки в test (или None, если её нет)

train = pd.read_csv(TRAIN_PATH, sep=SEP)
test  = pd.read_csv(TEST_PATH,  sep=SEP)

feature_cols = [c for c in train.columns if c != TARGET_COL and c in test.columns]

X_raw = train[feature_cols].copy()
y = train[TARGET_COL].values
X_test_raw = test[feature_cols].copy()

X_raw.shape, X_test_raw.shape


((41105, 285), (9276, 285))

In [49]:
fg = AutoMLPipelineFeatureGenerator(
    enable_numeric_features=True,
    enable_categorical_features=True,
    enable_datetime_features=True,
    enable_text_special_features=True,
    enable_text_ngram_features=True,
    enable_raw_text_features=False,
)

X_tr_feat = fg.fit_transform(X=X_raw, y=y)
X_te_feat = fg.transform(X_test_raw)

X_tr_feat.shape, X_te_feat.shape


Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    437899.84 MB
	Train Data (Original)  Memory Usage: 101.85 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerator...
		Fitting TextSpecialFeatureGenerator...
			Fitting BinnedFeatureGenerator...
			Fitting DropDuplicatesFeatureGenerator...
		Fitting TextNgramFeatureGenerator...
			Fitting CountVectorizer for text features: ['address']
			CountVectorizer fit with vocabulary size = 792
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Types of features in original data

((41105, 916), (9276, 916))

In [50]:
train_ag = X_tr_feat.copy()
train_ag[TARGET_COL] = y

test_ag = X_te_feat.copy()

train_ag.shape, test_ag.shape


((41105, 917), (9276, 916))

In [51]:
hyperparams_base = {
    "GBM": [
        {"extra_trees": False, "ag_args": {"name_suffix": "GBM"}},
        {"extra_trees": True,  "ag_args": {"name_suffix": "GBM_XT"}},
    ],
    "CAT": {
        "iterations": 4000,
        "depth": 8,
        "learning_rate": 0.01,
    },
    "XGB": {},
    "NN_TORCH": {},
}

predictor = TabularPredictor(
    label=TARGET_COL,
    problem_type=CONFIG.get("task_type", "regression"),
    eval_metric=CONFIG.get("metric", "rmse"),
).fit(
    train_data=train_ag,
    time_limit=1800,
    presets="medium_quality_faster_train",
    hyperparameters=hyperparams_base,
    ag_args_fit={"num_gpus": 1},
)


No path specified. Models will be saved in: "AutogluonModels/ag-20251116_215837"
Preset alias specified: 'medium_quality_faster_train' maps to 'medium_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #52~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Mon Dec  9 15:00:52 UTC 2
CPU Count:          62
Memory Avail:       427.95 GB / 503.46 GB (85.0%)
Disk Space Avail:   127.07 GB / 130.00 GB (97.7%)
Presets specified: ['medium_quality_faster_train']
Beginning AutoGluon training ... Time limit = 1800s
AutoGluon will save models to "/workspace/121/AutogluonModels/ag-20251116_215837"
Train Data Rows:    41105
Train Data Columns: 916
Label Column:       target
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:            

In [52]:
hyperparams_extra = {
    "CAT": {
        "iterations": 4000,
        "depth": 8,
        "learning_rate": 0.02,
    },
    "GBM": [
        {"extra_trees": False, "ag_args": {"name_suffix": "GBM_extra"}},
        {"extra_trees": True,  "ag_args": {"name_suffix": "GBM_XT_extra"}},
    ],
}

predictor = predictor.fit_extra(
    hyperparameters=hyperparams_extra,
    ag_args_fit={"num_gpus": 1},
    time_limit=1200,
)


Fitting 3 L1 models, fit_strategy="sequential" ...
Fitting model: LightGBMGBM_extra ... Training model for up to 1200.00s of the 1200.00s of remaining time.
		`import lightgbm` failed. A quick tip is to install via `pip install autogluon.tabular[lightgbm]==1.4.0`.
Fitting model: LightGBMGBM_XT_extra ... Training model for up to 1199.56s of the 1199.56s of remaining time.
		`import lightgbm` failed. A quick tip is to install via `pip install autogluon.tabular[lightgbm]==1.4.0`.
Fitting model: CatBoost_2 ... Training model for up to 1199.11s of the 1199.11s of remaining time.
	Fitting with cpus=62, gpus=1, mem=2.5/427.6 GB
	Training CatBoost_2 with GPU, note that this may negatively impact model quality compared to CPU training.
	-0.9797	 = Validation score   (-root_mean_squared_error)
	30.82s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: WeightedEnsemble_2_L2 ... Training model for up to 360.00s of the 1168.20s of remaining time.
	Ensemble Weights: {'CatBoost_2': 1.0

In [53]:
lb = predictor.leaderboard(train_ag, silent=True)
lb


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost,-0.999949,-0.978940,root_mean_squared_error,0.148261,0.051061,63.012222,0.148261,0.051061,63.012222,1,True,1
1,WeightedEnsemble_L2,-0.999949,-0.978940,root_mean_squared_error,0.151422,0.051380,63.014216,0.003161,0.000319,0.001994,2,True,2
2,CatBoost_2,-1.003683,-0.979668,root_mean_squared_error,0.102016,0.051478,30.824694,0.102016,0.051478,30.824694,1,True,3
3,WeightedEnsemble_2_L2,-1.003683,-0.979668,root_mean_squared_error,0.104718,0.051771,30.826800,0.002702,0.000293,0.002107,2,True,4


In [54]:
fi = predictor.feature_importance(train_ag)
fi.head(20)


Computing feature importance via permutation shuffling for 916 features using 5000 rows with 5 shuffle sets...
	1571.19s	= Expected runtime (314.24s per shuffle set)
	133.35s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
category,0.190031,0.003282,1.067495e-08,5,0.196789,0.183273
name,0.095356,0.005808,1.643011e-06,5,0.107314,0.083398
id,0.011836,0.002606,2.648099e-04,5,0.017202,0.006469
coordinates,0.008139,0.001545,1.486646e-04,5,0.011320,0.004957
address,0.005589,0.001491,5.540305e-04,5,0.008659,0.002519
__nlp__.москва,0.000860,0.000214,4.270179e-04,5,0.001301,0.000418
mean_income_1000m,0.000773,0.000084,1.655252e-05,5,0.000946,0.000600
__nlp__.троицк,0.000737,0.000363,5.234521e-03,5,0.001484,-0.000010
__nlp__.зеленоград,0.000690,0.000302,3.444998e-03,5,0.001311,0.000069
mean_income_300m,0.000645,0.000161,4.294521e-04,5,0.000977,0.000314


In [55]:
important_features = fi[fi["importance"] > 0].index.tolist()
len(important_features)


726

In [56]:
train_ag_imp = train_ag[important_features + [TARGET_COL]].copy()
test_ag_imp  = test_ag[important_features].copy()

train_ag_imp.shape, test_ag_imp.shape


((41105, 727), (9276, 726))

In [57]:
hyperparams_slim = {
    "CAT": {
        "iterations": 4000,
        "depth": 7,
        "learning_rate": 0.01,
    },
    "GBM": [
        {"extra_trees": False, "ag_args": {"name_suffix": "GBM_slim"}},
    ],
}

predictor_slim = TabularPredictor(
    label=TARGET_COL,
    problem_type=CONFIG.get("task_type", "regression"),
    eval_metric=CONFIG.get("metric", "rmse"),
).fit(
    train_data=train_ag_imp,
    time_limit=1200,
    presets="medium_quality_faster_train",
    hyperparameters=hyperparams_slim,
    ag_args_fit={"num_gpus": 1},
)

No path specified. Models will be saved in: "AutogluonModels/ag-20251116_220420"
Preset alias specified: 'medium_quality_faster_train' maps to 'medium_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #52~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Mon Dec  9 15:00:52 UTC 2
CPU Count:          62
Memory Avail:       423.48 GB / 503.46 GB (84.1%)
Disk Space Avail:   126.92 GB / 130.00 GB (97.6%)
Presets specified: ['medium_quality_faster_train']
Beginning AutoGluon training ... Time limit = 1200s
AutoGluon will save models to "/workspace/121/AutogluonModels/ag-20251116_220420"
Train Data Rows:    41105
Train Data Columns: 726
Label Column:       target
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:            

In [58]:
test_pred_base = predictor.predict(test_ag)
test_pred_slim = predictor_slim.predict(test_ag_imp)

test_pred_blend = 0.5 * test_pred_base.values + 0.5 * test_pred_slim.values


In [59]:
if ID_COL is not None and ID_COL in test.columns:
    sub_base = test[[ID_COL]].copy()
    sub_slim = test[[ID_COL]].copy()
    sub_blend = test[[ID_COL]].copy()
else:
    sub_base = pd.DataFrame()
    sub_slim = pd.DataFrame()
    sub_blend = pd.DataFrame()

sub_base[TARGET_COL] = test_pred_base
sub_slim[TARGET_COL] = test_pred_slim
sub_blend[TARGET_COL] = test_pred_blend

sub_base.to_csv("submission_ag_base.csv", index=False)
sub_slim.to_csv("submission_ag_slim.csv", index=False)
sub_blend.to_csv("submission_ag_blend.csv", index=False)

sub_blend.head()


,id,target
0,21472,3.402225
1,9837,3.115150
2,41791,3.925786
3,18441,3.096871
4,49348,2.726351


In [ ]:
'''сейчас попробую взять еще толще модели, а то скор сильно не меняется'''

In [60]:
heavy_hps = {
    "GBM": [
        {
            "extra_trees": False,
            "ag_args": {"name_suffix": "GBM_main"},
            "num_leaves": 64,
            "feature_fraction": 0.9,
            "bagging_fraction": 0.9,
            "bagging_freq": 1,
        },
        {
            "extra_trees": True,
            "ag_args": {"name_suffix": "GBM_xt"},
            "num_leaves": 128,
            "feature_fraction": 0.8,
            "bagging_fraction": 0.8,
            "bagging_freq": 1,
        },
    ],
    "CAT": [
        {
            "iterations": 6000,
            "depth": 8,
            "learning_rate": 0.03,
            "l2_leaf_reg": 3.0,
            "border_count": 254,
            "random_strength": 0.5,
            "bagging_temperature": 0.8,
            "ag_args": {"name_suffix": "CAT_main"},
        },
        {
            "iterations": 8000,
            "depth": 10,
            "learning_rate": 0.02,
            "l2_leaf_reg": 4.0,
            "border_count": 254,
            "random_strength": 0.3,
            "bagging_temperature": 0.5,
            "ag_args": {"name_suffix": "CAT_deep"},
        },
    ],
    "XGB": {
        "max_depth": 8,
        "subsample": 0.9,
        "colsample_bytree": 0.9,
        "n_estimators": 4000,
        "learning_rate": 0.03,
    },
    "NN_TORCH": {
        "ag_args": {"name_suffix": "NN_heavy"},
        "layers": "128-64-64",
        "dropout_prob": 0.1,
    },
}


In [61]:
from autogluon.tabular import TabularPredictor

heavy_time_limit = CONFIG.get("heavy_time_limit_sec", 7200)  # 2 часа, под задачу можно меньше/больше

predictor_heavy = TabularPredictor(
    label=TARGET_COL,
    problem_type=CONFIG.get("task_type", "regression"),
    eval_metric=CONFIG.get("metric", "rmse"),
).fit(
    train_data=train_ag,
    time_limit=heavy_time_limit,
    presets="best_quality",          # heavy режим
    num_bag_folds=5,                 # бэггинг по фолдам
    num_bag_sets=2,                  # повторение бэггинга
    num_stack_levels=2,              # двухуровневый stacking
    hyperparameters=heavy_hps,
    ag_args_fit={"num_gpus": 1},
)


No path specified. Models will be saved in: "AutogluonModels/ag-20251116_221138"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #52~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Mon Dec  9 15:00:52 UTC 2
CPU Count:          62
Memory Avail:       427.52 GB / 503.46 GB (84.9%)
Disk Space Avail:   126.78 GB / 130.00 GB (97.5%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=2, num_bag_folds=5, num_bag_sets=2
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of 

In [62]:
extra_hps_heavy = {
    "CAT": {
        "iterations": 8000,
        "depth": 8,
        "learning_rate": 0.008,
        "l2_leaf_reg": 3.0,
        "ag_args": {"name_suffix": "CAT_extra"},
    },
    "GBM": [
        {
            "extra_trees": False,
            "ag_args": {"name_suffix": "GBM_extra"},
            "num_leaves": 128,
            "feature_fraction": 0.85,
            "bagging_fraction": 0.85,
            "bagging_freq": 1,
        },
    ],
}

predictor_heavy = predictor_heavy.fit_extra(
    hyperparameters=extra_hps_heavy,
    ag_args_fit={"num_gpus": 1},
    time_limit=1800,
)


Fitting 2 L1 models, fit_strategy="sequential" ...
Fitting model: LightGBMGBM_extra_BAG_L1 ... Training model for up to 1800.00s of the 1800.00s of remaining time.
		`import lightgbm` failed. A quick tip is to install via `pip install autogluon.tabular[lightgbm]==1.4.0`.
Fitting model: CatBoostCAT_extra_BAG_L1 ... Training model for up to 1799.58s of the 1799.58s of remaining time.
	Fitting 10 child models (S1F1 - S2F5) | Fitting with SequentialLocalFoldFittingStrategy (sequential: cpus=62, gpus=1)
	Training S1F1 with GPU, note that this may negatively impact model quality compared to CPU training.
	Training S1F2 with GPU, note that this may negatively impact model quality compared to CPU training.
	Training S1F3 with GPU, note that this may negatively impact model quality compared to CPU training.
	Training S1F4 with GPU, note that this may negatively impact model quality compared to CPU training.
	Training S1F5 with GPU, note that this may negatively impact model quality compared to 

In [63]:
refit_models = predictor_heavy.refit_full()  # вернёт список имён моделей
refit_models


Refitting models via `predictor.refit_full` using all of the data (combined train and validation)...
	Models trained in this way will have the suffix "_FULL" and have NaN validation score.
	This process is not bound by time_limit, but should take less time than the original `predictor.fit` call.
	To learn more, refer to the `.refit_full` method docstring which explains how "_FULL" models differ from normal models.
Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: CatBoostCAT_main_BAG_L1_FULL ...
	Training S1F1 with GPU, note that this may negatively impact model quality compared to CPU training.
	47.04s	 = Training   runtime
Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: CatBoostCAT_deep_BAG_L1_FULL ...
	Training S1F1 with GPU, note that this may negatively impact model quality compared to CPU training.
	155.07s	 = Training   runtime
Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: CatBoostCAT_extra_BAG_L1_FULL ...
	Training S1F1 with

{'CatBoostCAT_main_BAG_L1': 'CatBoostCAT_main_BAG_L1_FULL',
 'CatBoostCAT_deep_BAG_L1': 'CatBoostCAT_deep_BAG_L1_FULL',
 'CatBoostCAT_extra_BAG_L1': 'CatBoostCAT_extra_BAG_L1_FULL',
 'WeightedEnsemble_L2': 'WeightedEnsemble_L2_FULL',
 'CatBoostCAT_main_BAG_L2': 'CatBoostCAT_main_BAG_L2_FULL',
 'CatBoostCAT_deep_BAG_L2': 'CatBoostCAT_deep_BAG_L2_FULL',
 'WeightedEnsemble_2_L2': 'WeightedEnsemble_2_L2_FULL',
 'WeightedEnsemble_L3': 'WeightedEnsemble_L3_FULL',
 'CatBoostCAT_main_BAG_L3': 'CatBoostCAT_main_BAG_L3_FULL',
 'CatBoostCAT_deep_BAG_L3': 'CatBoostCAT_deep_BAG_L3_FULL',
 'WeightedEnsemble_L4': 'WeightedEnsemble_L4_FULL'}

In [64]:
lb_heavy = predictor_heavy.leaderboard(train_ag, silent=True)
lb_heavy


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoostCAT_deep_BAG_L3_FULL,-0.916279,NaN,root_mean_squared_error,0.599813,NaN,225.043656,0.099124,NaN,8.463934,3,True,21
1,CatBoostCAT_main_BAG_L3_FULL,-0.923221,NaN,root_mean_squared_error,0.584995,NaN,221.072683,0.084306,NaN,4.492962,3,True,20
2,CatBoostCAT_deep_BAG_L2_FULL,-0.923662,NaN,root_mean_squared_error,0.411435,NaN,211.649469,0.100992,NaN,9.546101,2,True,17
3,WeightedEnsemble_L3_FULL,-0.924691,NaN,root_mean_squared_error,0.501887,NaN,216.589046,0.001198,NaN,0.009325,3,True,19
4,WeightedEnsemble_L4_FULL,-0.925978,NaN,root_mean_squared_error,0.413476,NaN,211.667761,0.002040,NaN,0.018292,4,True,22
5,CatBoostCAT_main_BAG_L2_FULL,-0.935022,NaN,root_mean_squared_error,0.399697,NaN,207.033620,0.089254,NaN,4.930252,2,True,16
6,CatBoostCAT_deep_BAG_L3,-0.943104,-1.020547,root_mean_squared_error,2.916770,3.570393,3347.472876,0.467275,0.723779,460.999281,3,True,8
7,CatBoostCAT_main_BAG_L3,-0.944375,-1.020356,root_mean_squared_error,2.881808,3.521692,3041.426079,0.432313,0.675078,154.952484,3,True,7
8,CatBoostCAT_deep_BAG_L2,-0.946427,-1.019281,root_mean_squared_error,2.014383,2.123481,2710.522289,0.443499,0.746373,518.578130,2,True,5
9,WeightedEnsemble_L3,-0.946615,-1.019280,root_mean_squared_error,2.451519,2.847040,2886.482920,0.002024,0.000426,0.009325,3,True,6


In [65]:
fi_heavy = predictor_heavy.feature_importance(train_ag)
fi_heavy.head(30)


Computing feature importance via permutation shuffling for 916 features using 5000 rows with 5 shuffle sets...
	1222.19s	= Expected runtime (244.44s per shuffle set)
	160.98s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
category,0.256087,0.006250,4.254124e-08,5,0.268956,0.243218
name,0.143222,0.009931,2.756245e-06,5,0.163670,0.122774
id,0.026039,0.003964,6.249724e-05,5,0.034201,0.017877
coordinates,0.009665,0.001618,9.073447e-05,5,0.012996,0.006335
address,0.007237,0.001181,8.226482e-05,5,0.009669,0.004805
mean_income_1000m,0.004080,0.000457,1.859026e-05,5,0.005022,0.003139
mean_income_300m,0.004065,0.000653,7.712692e-05,5,0.005409,0.002721
__nlp__.москва,0.003000,0.000750,4.327484e-04,5,0.004545,0.001455
address.symbol_count..,0.002978,0.000774,5.027841e-04,5,0.004572,0.001383
homes_300m,0.002896,0.000545,1.432752e-04,5,0.004017,0.001775


In [66]:
test_pred_heavy = predictor_heavy.predict(test_ag)
test_pred_heavy.head()


0    3.433025
1    3.425081
2    4.002472
3    3.274601
4    2.883332
Name: target, dtype: float32

In [67]:
# Предсказания medium (если predictor ещё есть)
try:
    test_pred_medium = predictor.predict(test_ag)
except NameError:
    test_pred_medium = None

if ID_COL is not None and ID_COL in test.columns:
    sub_heavy = test[[ID_COL]].copy()
    sub_blend = test[[ID_COL]].copy()
else:
    sub_heavy = pd.DataFrame()
    sub_blend = pd.DataFrame()

sub_heavy[TARGET_COL] = test_pred_heavy

if test_pred_medium is not None:
    test_pred_blend = 0.5 * test_pred_heavy.values + 0.5 * test_pred_medium.values
    sub_blend[TARGET_COL] = test_pred_blend
else:
    sub_blend[TARGET_COL] = test_pred_heavy.values

sub_heavy.to_csv("submission_ag_heavy.csv", index=False)
sub_blend.to_csv("submission_ag_heavy_blend.csv", index=False)

sub_heavy.head()


,id,target
0,21472,3.433025
1,9837,3.425081
2,41791,4.002472
3,18441,3.274601
4,49348,2.883332
